# Fair Binary Classification with SearchFair on CelebA and Adult

Here, we show how to use SearchFair on two datasets: CelebA and Adult

## Imports

We start by importing SearchFair from the installed package.

In [1]:
from searchfair import SearchFair

Second, we load some necessary methods and numpy.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

# We are ignoring cvxpys warning about disciplined programming rules. 
import warnings
warnings.filterwarnings('ignore')

# The CelebA dataset

On the Celebrity Faces dataset we are given descriptions of celebrity faces, with 40 binary attributes. Here, we use the Attribute 'Smiling' as the class label, and sex as the sensitive attribute. 

In [3]:
import get_real_data as get_data

# Load Data
x_data, y_data, s_data = get_data.get_celebA_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1000
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)

Here are some basic information about the dataset:

In [4]:
import utils as ut
ut.print_data_stats(s_data, y_data)

Total data points: 202599
# non-protected examples: 118165
# protected examples: 84434
# non-protected examples in positive class: 63871 (54.1%)
# protected examples in positive class: 33798 (40.0%)


## Creating a SearchFair Model

### Demographic Parity

To learn a classifier with SearchFair, we need to choose a kernel between 'linear' and 'rbf', and we need to choose a fairness notion - either Demographic Parity (DDP) or Equality of Opportunity (DEO). 

In [5]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress

Let us choose a regularization parameter and then fit a model with the default settings. 

In [6]:
# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)

Preprocessing...
Compilation of CVXPY (might take a while for v1.1)
Testing lambda_min: 0.00
Obtained: DDP = 0.2052 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.7897 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = -0.5074 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DDP = -0.1176 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DDP = 0.2052 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DDP = 0.0672 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.2188
Obtained: DDP = -0.0127 with lambda = 0.2188
----------Iteration #5----------
Testing new Lambda: 0.2031
Obtained: DDP = 0.0502 with lambda = 0.2031
----------Iteration #6----------
Testing new Lambda: 0.2109
Obtained: DDP = -0.0103 with lambda = 0.2109
----------Iteration #7----------
Te

SearchFair(reg_beta=0.0001, verbose=True)

To print out the Accuracy and the fairness notions Demographic Parity and Equality of Opportuniy, we define the following function. 

In [7]:
def print_clf_stats(model, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

Now lets see, if we obtained a fair classifier with respect to the fairness notions we specified. 

In [8]:
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 82.3000%
DDP: 0.9500% DEO: 10.0143%
----------Test----------
Accuracy: 81.4528%
DDP: 7.1903% DEO: 11.6502%


Probably, you will see that the train DDP is small, and SearchFair succeed to find a fair classifier. The test DDP might or might not be close to 0. This is due to the small number of points which are used to reduce running time for this example notebook. Go ahead and try it with more points!

### Equality of Opportunity

Now we try the same using a more complex rbf kernel, and we try to improve Equality of Opportunity this time. 

In [9]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Compilation of CVXPY (might take a while for v1.1)
Testing lambda_min: 0.00
Obtained: DEO = 0.0981 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.9073 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = -0.8149 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = -0.0438 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.0394 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DEO = 0.0168 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.2188
Obtained: DEO = 0.0116 with lambda = 0.2188
----------Iteration #5----------
Testing new Lambda: 0.2344
Obtained: DEO = -0.0072 with lambda = 0.2344
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.2344 with fairness -0.0072----------
---------

### Cross Validation - GridSearch

It is also possible to use GridSearchCV for the regularization paramter beta, and, if used, the width of the rbf kernel. But running this might take some time.

In [10]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model,cv_params, cv=3, verbose=2, n_jobs=1, scoring='accuracy', refit=True)
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] gamma=0.01, reg_beta=0.0001 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... gamma=0.01, reg_beta=0.0001, total= 1.0min
[CV] gamma=0.01, reg_beta=0.0001 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


[CV] ...................... gamma=0.01, reg_beta=0.0001, total= 1.0min
[CV] gamma=0.01, reg_beta=0.0001 .....................................
[CV] ...................... gamma=0.01, reg_beta=0.0001, total=  56.1s
[CV] gamma=0.01, reg_beta=0.001 ......................................
[CV] ....................... gamma=0.01, reg_beta=0.001, total= 1.0min
[CV] gamma=0.01, reg_beta=0.001 ......................................
[CV] ....................... gamma=0.01, reg_beta=0.001, total=  52.4s
[CV] gamma=0.01, reg_beta=0.001 ......................................
[CV] ....................... gamma=0.01, reg_beta=0.001, total=  57.2s
[CV] gamma=0.01, reg_beta=0.01 .......................................
[CV] ........................ gamma=0.01, reg_beta=0.01, total=  58.8s
[CV] gamma=0.01, reg_beta=0.01 .......................................
[CV] ........................ gamma=0.01, reg_beta=0.01, total=  51.5s
[CV] gamma=0.01, reg_beta=0.01 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 24.3min finished


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf'), n_jobs=1,
             param_grid={'gamma': [0.01, 0.02631578947368421, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=2)

In [11]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 84.8000%
DDP: 0.0982% DEO: 4.0297%
----------Test----------
Accuracy: 83.7901%
DDP: 6.6078% DEO: 4.0772%


## Adult dataset

In the fairness literature, the adult dataset is a very popular dataset. It contains US census data from 1994, where the class label indicates if the income is higher or lower than 50.000$. The binary sensitive attribute here, is the sex.

In [12]:
# Load Data
x_data, y_data, s_data = get_data.get_adult_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1000
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
ut.print_data_stats(s_data, y_data)

Total data points: 48842
# non-protected examples: 32650
# protected examples: 16192
# non-protected examples in positive class: 9918 (30.4%)
# protected examples in positive class: 1769 (10.9%)


If you want, you can also try SearchFair on Adult. 

In [14]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model_adult = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model_adult,cv_params, cv=3, verbose=2, n_jobs=1, scoring='accuracy')
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] gamma=0.01, reg_beta=0.0001 .....................................
Preprocessing...
Compilation of CVXPY (might take a while for v1.1)
Testing lambda_min: 0.00


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Obtained: DDP = 0.2249 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.3774 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------
[CV] ...................... gamma=0.01, reg_beta=0.0001, total=  57.0s
[CV] gamma=0.01, reg_beta=0.0001 .....................................
Preprocessing...
Compilation of CVXPY (might take a while for v1.1)
Testing lambda_min: 0.00


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   57.0s remaining:    0.0s


Obtained: DDP = 0.2638 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.4804 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------
[CV] ...................... gamma=0.01, reg_beta=0.0001, total=  57.7s
[CV] gamma=0.01, reg_beta=0.0001 .....................................
Preprocessing...
Compilation of CVXPY (might take a while for v1.1)
Testing lambda_min: 0.00
Obtained: DDP = 0.2202 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.2683 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------
[CV] ............

Obtained: DDP = 0.2638 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.3796 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------
[CV] ......... gamma=0.05555555555555555, reg_beta=0.01, total=  59.8s
[CV] gamma=0.05555555555555555, reg_beta=0.01 ........................
Preprocessing...
Compilation of CVXPY (might take a while for v1.1)
Testing lambda_min: 0.00
Obtained: DDP = 0.2148 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = 0.0000 with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
----------Found Lambda 1.0000 with fairness 0.0000----------
[CV] ......... gamma=0.05555555555555555, reg_beta=0.01, total=  48.9s
[CV] gamma=0.1, reg_beta=0.0001 ......................................
Preprocessing...
Compilation

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 26.2min finished


Obtained: DDP = 0.2365 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.2827 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf', verbose=True), n_jobs=1,
             param_grid={'gamma': [0.01, 0.05555555555555555, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=2)

In [15]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 74.8000%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 76.0984%
DDP: 0.0000% DEO: 0.0000%
